In [ ]:
#####better sementation, cell body, outside but not axon is good##################

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, meijering, threshold_local
import napari
import pandas as pd
import imageio
import cv2

from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, skeletonize_3d
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries  # <-- added

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (axon + cell body)

# --------------------------
# 2) CH1: blobs (lysosomes)
# --------------------------
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# Per-blob DF & region bins (optional export kept)
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("volume_voxels", "sum")
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 3) Viewer base
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 4) CH2: neuron segmentation (axon = thin, cell body = thick)  ✅ three-way
# --------------------------
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)

# Thin-process ridge response (neurites)
ridge = meijering(ch2, sigmas=np.linspace(0.6, 2.0, 8), black_ridges=False)

# (1) Base NEURON mask (axon + body) via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=81, offset=-0.4*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(3))

# (2) CELL BODY (thick) via distance transform
dist = edt(neuron_mask)
cell_body_min_radius_vox = 1   # tune (e.g., 4–6) for thicker soma
cell_body_mask = (dist >= cell_body_min_radius_vox)
cell_body_mask &= neuron_mask
cell_body_mask = binary_opening(cell_body_mask, ball(3))
cell_body_mask = binary_closing(cell_body_mask, ball(4))

# Label ALL soma (for watershed seeding) and count cells
body_lab = label(cell_body_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cell bodies.")

# Watershed to partition the entire neuron into per-cell territories
if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

# (3) AXON (thin) via ridge + local threshold, inside neuron, excluding cell body
axon_candidate = np.zeros_like(ridge, dtype=bool)
for z in range(ridge.shape[0]):
    Rt = threshold_local(ridge[z], block_size=81, offset=-0.4*np.std(ridge[z]))
    axon_candidate[z] = ridge[z] > Rt
axon_candidate = remove_small_objects(axon_candidate, min_size=100, connectivity=3)
axon_candidate = binary_opening(axon_candidate, ball(1))
axon_candidate = binary_closing(axon_candidate, ball(1))

axon_mask = axon_candidate & neuron_mask & (~cell_body_mask)   # exclusive split

# (4) Optional: skeleton for the axon
axon_skel = skeletonize_3d(axon_mask)

# (5) Body shape (circular/elliptical), measured on largest cell body slice (kept scaffolding)
axon_shape = "unknown"
if body_lab.max() > 0:
    props3d = regionprops(body_lab)
    props3d.sort(key=lambda p: p.area, reverse=True)
    lab = props3d[0].label
    z_areas = [np.sum(body_lab[z] == lab) for z in range(body_lab.shape[0])]
    z_best = int(np.argmax(z_areas))
    body_2d = (body_lab[z_best] == lab).astype(np.uint8)
    rp2 = regionprops(body_2d)
    # if rp2:
    #     ecc = rp2[0].eccentricity
    #     axon_shape = "circular" if ecc < 0.6 else "elliptical"

# Sanity
print("neuron voxels:", int(neuron_mask.sum()))
print("cell body voxels:", int(cell_body_mask.sum()))
print("axon voxels:", int(axon_mask.sum()))

# --------------------------
# 5) Map lysosomes to specific cell IDs and parts (axon/body)
# --------------------------
in_neuron = []
in_axon_ch2 = []
in_body_ch2 = []
location_ch2 = []
cell_id_list = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            in_neuron.append(False); in_axon_ch2.append(False); in_body_ch2.append(False)
            location_ch2.append("outside"); cell_id_list.append(0)
            continue

        n  = bool(neuron_mask[zz, yy, xx])
        ax = bool(axon_mask[zz, yy, xx])
        cb = bool(cell_body_mask[zz, yy, xx])
        cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0

        in_neuron.append(n)
        in_axon_ch2.append(ax)
        in_body_ch2.append(cb)

        if not n:
            location_ch2.append("outside")
            cell_id_list.append(0)
        else:
            if cb:
                location_ch2.append("cell body")
            elif ax:
                location_ch2.append("axon")
            else:
                location_ch2.append("neuron_other")
            cell_id_list.append(cid)
else:
    # no blobs -> empty lists
    pass

if len(df) > 0:
    df["in_neuron_ch2"] = in_neuron
    df["in_axon_ch2"]   = in_axon_ch2
    df["in_body_ch2"]   = in_body_ch2
    df["location_ch2"]  = location_ch2
    df["cell_id_ch2"]   = cell_id_list  # 0 = outside / unassigned

    # overall counts
    counts = pd.Series({
        "total_lysosomes": len(df),
        "in_neuron": int(np.sum(df["in_neuron_ch2"])),
        "in_axon":   int(np.sum(df["in_axon_ch2"])),
        "in_body":   int(np.sum(df["in_body_ch2"])),
    })
    counts.to_frame(name="value").to_csv("lysosome_location_summary.csv")

    # per-class counts
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_by_location.csv", index=False)

    # per-cell, per-part counts
    per_cell_counts = (
        df.groupby(["cell_id_ch2", "location_ch2"])
          .size()
          .reset_index(name="count")
          .sort_values(["cell_id_ch2", "location_ch2"])
    )
    per_cell_counts.to_csv("lysosome_counts_by_cell_and_part.csv", index=False)

    # flat summary per cell
    summary_per_cell = (
        df.pivot_table(index="cell_id_ch2",
                       columns="location_ch2",
                       values="id",
                       aggfunc="count",
                       fill_value=0)
        .reset_index()
        .rename(columns={"cell_id_ch2": "cell_id"})
    )
    summary_per_cell.to_csv("lysosome_summary_per_cell.csv", index=False)

    # Export full table
    df.to_csv("lysosomes_with_cell_ids.csv", index=False)

print("Saved: lysosome_location_summary.csv, lysosome_counts_by_location.csv, "
      "lysosome_counts_by_cell_and_part.csv, lysosome_summary_per_cell.csv, lysosomes_with_cell_ids.csv")

# --------------------------
# 6) Visualization
# --------------------------
# Axon (red)
axon_layer = viewer.add_labels(axon_mask.astype(np.uint8), name='Axon (Ch2)', opacity=0.35)
try:
    axon_layer.color = {1: (1.0, 0.0, 0.0, 1.0)}  # red
except Exception:
    pass
axon_layer.blending = 'translucent_no_depth'

# Cell body (green)
cellbody_layer = viewer.add_labels(cell_body_mask.astype(np.uint8), name='Cell body (Ch2)', opacity=0.35)
try:
    cellbody_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cellbody_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Axon skeleton (thin gray)
viewer.add_image(
    axon_skel.astype(np.uint8),
    name='Axon skeleton',
    blending='additive',
    contrast_limits=(0, 1),
    colormap='gray',
    opacity=0.8
)

# Lysosomes colored by location (orange=axon, cyan=cell body, white=outside, gray=neuron_other)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist())
    colors = np.zeros((len(loc), 4), dtype=float)
    colors[loc == "axon"]         = [1.0, 0.5, 0.0, 1.0]  # orange
    colors[loc == "cell body"]    = [0.0, 1.0, 1.0, 1.0]  # cyan
    colors[loc == "outside"]      = [1.0, 1.0, 1.0, 1.0]  # white
    colors[loc == "neuron_other"] = [0.6, 0.6, 0.6, 1.0]  # gray

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        face_color=colors,
        name='Lysosomes (orange=axon, cyan=cell body)'
    )
    try:
        pts.properties = {
            'lys_id': df['id'].to_numpy(),
            'where':  df['location_ch2'].to_numpy(),
            'cell':   df['cell_id_ch2'].to_numpy()
        }
        pts.text = {'text': 'ID:{lys_id} C:{cell} {where}', 'size': 8, 'color': 'white', 'anchor': 'upper left'}
        pts.edge_color = 'black'
        pts.edge_width = 0.3
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation body/axon (per-cell territories)"
except Exception:
    pass

# --------------------------
# 7) Quick fused video (optional)
# --------------------------
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    axon = (axon_mask[z].astype(np.uint8) * 255)
    body = (cell_body_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 2] = np.maximum(overlay[..., 2], axon)  # red for axon
    overlay[..., 1] = np.maximum(overlay[..., 1], body)  # green for cell body
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < axon_mask.shape[1] and 0 <= x < axon_mask.shape[2] and axon_mask[z, y, x]:
                color = (0, 128, 255)  # orange-ish (axon)
            elif 0 <= y < cell_body_mask.shape[1] and 0 <= x < cell_body_mask.shape[2] and cell_body_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell body)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_axon_cellbody.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_axon_cellbody.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_axon_cellbody.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_axon_cellbody.gif")

# --------------------------
# 8) Run viewer
# --------------------------
napari.run()

In [ ]:
#######version 1 + label

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, meijering, threshold_local
import napari
import pandas as pd
import imageio
import cv2

from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, skeletonize_3d
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries  # <-- added

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_2_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"


with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (axon + cell body)

# --------------------------
# 2) CH1: blobs (lysosomes)
# --------------------------
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# Per-blob DF & region bins (optional export kept)
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("volume_voxels", "sum")
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 3) Viewer base
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 4) CH2: neuron segmentation (axon = thin, cell body = thick)  ✅ three-way
# --------------------------
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)

# Thin-process ridge response (neurites)
ridge = meijering(ch2, sigmas=np.linspace(0.6, 2.0, 8), black_ridges=False)

# (1) Base NEURON mask (axon + body) via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=81, offset=-0.4*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(3))

# (2) CELL BODY (thick) via distance transform
dist = edt(neuron_mask)
cell_body_min_radius_vox = 1   # tune (e.g., 4–6) for thicker soma
cell_body_mask = (dist >= cell_body_min_radius_vox)
cell_body_mask &= neuron_mask
cell_body_mask = binary_opening(cell_body_mask, ball(3))
cell_body_mask = binary_closing(cell_body_mask, ball(4))

# Label ALL soma (for watershed seeding) and count cells
body_lab = label(cell_body_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cell bodies.")

# Watershed to partition the entire neuron into per-cell territories
if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

# (3) AXON (thin) via ridge + local threshold, inside neuron, excluding cell body
axon_candidate = np.zeros_like(ridge, dtype=bool)
for z in range(ridge.shape[0]):
    Rt = threshold_local(ridge[z], block_size=81, offset=-0.4*np.std(ridge[z]))
    axon_candidate[z] = ridge[z] > Rt
axon_candidate = remove_small_objects(axon_candidate, min_size=100, connectivity=3)
axon_candidate = binary_opening(axon_candidate, ball(1))
axon_candidate = binary_closing(axon_candidate, ball(1))

axon_mask = axon_candidate & neuron_mask & (~cell_body_mask)   # exclusive split

# (4) Optional: skeleton for the axon
axon_skel = skeletonize_3d(axon_mask)

# (5) Body shape (circular/elliptical), measured on largest cell body slice (kept scaffolding)
axon_shape = "unknown"
if body_lab.max() > 0:
    props3d = regionprops(body_lab)
    props3d.sort(key=lambda p: p.area, reverse=True)
    lab = props3d[0].label
    z_areas = [np.sum(body_lab[z] == lab) for z in range(body_lab.shape[0])]
    z_best = int(np.argmax(z_areas))
    body_2d = (body_lab[z_best] == lab).astype(np.uint8)
    rp2 = regionprops(body_2d)
    # if rp2:
    #     ecc = rp2[0].eccentricity
    #     axon_shape = "circular" if ecc < 0.6 else "elliptical"

# Sanity
print("neuron voxels:", int(neuron_mask.sum()))
print("cell body voxels:", int(cell_body_mask.sum()))
print("axon voxels:", int(axon_mask.sum()))

# --------------------------
# 5) Map lysosomes to specific cell IDs and parts (axon/body)
# --------------------------
in_neuron = []
in_axon_ch2 = []
in_body_ch2 = []
location_ch2 = []
cell_id_list = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            in_neuron.append(False); in_axon_ch2.append(False); in_body_ch2.append(False)
            location_ch2.append("outside"); cell_id_list.append(0)
            continue

        n  = bool(neuron_mask[zz, yy, xx])
        ax = bool(axon_mask[zz, yy, xx])
        cb = bool(cell_body_mask[zz, yy, xx])
        cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0

        in_neuron.append(n)
        in_axon_ch2.append(ax)
        in_body_ch2.append(cb)

        if not n:
            location_ch2.append("outside")
            cell_id_list.append(0)
        else:
            if cb:
                location_ch2.append("cell body")
            elif ax:
                location_ch2.append("axon")
            else:
                location_ch2.append("neuron_other")
            cell_id_list.append(cid)
else:
    # no blobs -> empty lists
    pass

if len(df) > 0:
    df["in_neuron_ch2"] = in_neuron
    df["in_axon_ch2"]   = in_axon_ch2
    df["in_body_ch2"]   = in_body_ch2
    df["location_ch2"]  = location_ch2
    df["cell_id_ch2"]   = cell_id_list  # 0 = outside / unassigned

    # overall counts
    counts = pd.Series({
        "total_lysosomes": len(df),
        "in_neuron": int(np.sum(df["in_neuron_ch2"])),
        "in_axon":   int(np.sum(df["in_axon_ch2"])),
        "in_body":   int(np.sum(df["in_body_ch2"])),
    })
    counts.to_frame(name="value").to_csv("lysosome_location_summary.csv")

    # per-class counts
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_by_location.csv", index=False)

    # per-cell, per-part counts
    per_cell_counts = (
        df.groupby(["cell_id_ch2", "location_ch2"])
          .size()
          .reset_index(name="count")
          .sort_values(["cell_id_ch2", "location_ch2"])
    )
    per_cell_counts.to_csv("lysosome_counts_by_cell_and_part.csv", index=False)

    # flat summary per cell
    summary_per_cell = (
        df.pivot_table(index="cell_id_ch2",
                       columns="location_ch2",
                       values="id",
                       aggfunc="count",
                       fill_value=0)
        .reset_index()
        .rename(columns={"cell_id_ch2": "cell_id"})
    )
    summary_per_cell.to_csv("lysosome_summary_per_cell.csv", index=False)

    # Export full table
    df.to_csv("lysosomes_with_cell_ids.csv", index=False)

print("Saved: lysosome_location_summary.csv, lysosome_counts_by_location.csv, "
      "lysosome_counts_by_cell_and_part.csv, lysosome_summary_per_cell.csv, lysosomes_with_cell_ids.csv")

# --------------------------
# 6) Visualization
# --------------------------
# Axon (red)
axon_layer = viewer.add_labels(axon_mask.astype(np.uint8), name='Axon (Ch2)', opacity=0.35)
try:
    axon_layer.color = {1: (1.0, 0.0, 0.0, 1.0)}  # red
except Exception:
    pass
axon_layer.blending = 'translucent_no_depth'

# Cell body (green)
cellbody_layer = viewer.add_labels(cell_body_mask.astype(np.uint8), name='Cell body (Ch2)', opacity=0.35)
try:
    cellbody_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cellbody_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Axon skeleton (thin gray)
viewer.add_image(
    axon_skel.astype(np.uint8),
    name='Axon skeleton',
    blending='additive',
    contrast_limits=(0, 1),
    colormap='gray',
    opacity=0.8
)

# Lysosomes colored by location (orange=axon, cyan=cell body, white=outside, gray=neuron_other)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist())
    colors = np.zeros((len(loc), 4), dtype=float)
    colors[loc == "axon"]         = [1.0, 0.5, 0.0, 1.0]  # orange
    colors[loc == "cell body"]    = [0.0, 1.0, 1.0, 1.0]  # cyan
    colors[loc == "outside"]      = [1.0, 1.0, 1.0, 1.0]  # white
    colors[loc == "neuron_other"] = [0.6, 0.6, 0.6, 1.0]  # gray

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        face_color=colors,
        name='Lysosomes (orange=axon, cyan=cell body)'
    )
    try:
        pts.properties = {
            'lys_id': df['id'].to_numpy(),
            'where':  df['location_ch2'].to_numpy(),
            'cell':   df['cell_id_ch2'].to_numpy()
        }
        pts.text = {'text': 'ID:{lys_id} C:{cell} {where}', 'size': 8, 'color': 'white', 'anchor': 'upper left'}
        pts.edge_color = 'black'
        pts.edge_width = 0.3
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation body/axon (per-cell territories)"
except Exception:
    pass

# --------------------------
# 6b) EXTRA: Labels for ALL cell bodies (with vs. without lysosomes)
# --------------------------
try:
    if len(df) > 0 and n_cells > 0 and body_lab.max() > 0:
        # find cell IDs with lysosomes in soma
        body_hits = df[(df["location_ch2"] == "cell body") & (df["cell_id_ch2"] > 0)]
        ids_with_lyso = set(np.unique(body_hits["cell_id_ch2"].to_numpy()))
        all_ids = set(range(1, int(body_lab.max()) + 1))
        ids_without_lyso = all_ids - ids_with_lyso

        props = regionprops(body_lab)
        pts_yes, labels_yes = [], []
        pts_no, labels_no   = [], []

        for prop in props:
            cid = int(prop.label)
            cz, cy, cx = prop.centroid  # (z, y, x)
            if cid in ids_with_lyso:
                pts_yes.append([cz, cy, cx])
                labels_yes.append(f"Cell {cid}")
            else:
                pts_no.append([cz, cy, cx])
                labels_no.append(f"Cell {cid}")

        # layer: soma labels with lysosomes (yellow)
        if pts_yes:
            lbl_yes = viewer.add_points(
                np.asarray(pts_yes, dtype=float),
                name="Soma labels (HAS lysosomes)",
                size=0.1,
                face_color=[0, 0, 0, 0],
                edge_color=[0, 0, 0, 0],
                edge_width=0
            )
            lbl_yes.text = {"text": labels_yes, "size": 28, "color": "yellow", "anchor": "center"}
            lbl_yes.blending = "translucent_no_depth"

        # layer: soma labels without lysosomes (light gray)
        if pts_no:
            lbl_no = viewer.add_points(
                np.asarray(pts_no, dtype=float),
                name="Soma labels (NO lysosomes)",
                size=0.1,
                face_color=[0, 0, 0, 0],
                edge_color=[0, 0, 0, 0],
                edge_width=0
            )
            lbl_no.text = {"text": labels_no, "size": 28, "color": "lightgray", "anchor": "center"}
            lbl_no.blending = "translucent_no_depth"

        # (optional) stamp IDs sparsely across territories
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES and cell_seg.max() > 0:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"

except Exception as e:
    print("Label overlay error:", e)

# --------------------------
# 7) Quick fused video (optional)
# --------------------------
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    axon = (axon_mask[z].astype(np.uint8) * 255)
    body = (cell_body_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 2] = np.maximum(overlay[..., 2], axon)  # red for axon
    overlay[..., 1] = np.maximum(overlay[..., 1], body)  # green for cell body
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < axon_mask.shape[1] and 0 <= x < axon_mask.shape[2] and axon_mask[z, y, x]:
                color = (0, 128, 255)  # orange-ish (axon)
            elif 0 <= y < cell_body_mask.shape[1] and 0 <= x < cell_body_mask.shape[2] and cell_body_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell body)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_axon_cellbody.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_axon_cellbody.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_axon_cellbody.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_axon_cellbody.gif")

# --------------------------
# 8) Run viewer
# --------------------------
napari.run()

In [ ]:
#######VERSION 2################################NOT GOOD EL VIEWER DESPUES DE EJECUTAR ES TODO NEGRO

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, meijering, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import time

from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, skeletonize_3d
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (axon + cell body)

# --------------------------
# 2) CH1: blobs (lysosomes)
# --------------------------
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# Per-blob DF & region bins (optional export kept)
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("volume_voxels", "sum")
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 3) Viewer base
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 4) CH2: neuron segmentation (axon = thin, cell body = thick)  ✅ three-way
# --------------------------
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)

# Thin-process ridge response (neurites)
ridge = meijering(ch2, sigmas=np.linspace(0.6, 2.0, 8), black_ridges=False)

# (1) Base NEURON mask (axon + body) via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=81, offset=-0.4*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(3))

# (2) CELL BODY (thick) via distance transform
dist = edt(neuron_mask)
cell_body_min_radius_vox = 1   # tune (e.g., 4–6) for thicker soma
cell_body_mask = (dist >= cell_body_min_radius_vox)
cell_body_mask &= neuron_mask
cell_body_mask = binary_opening(cell_body_mask, ball(3))
cell_body_mask = binary_closing(cell_body_mask, ball(4))

# Label ALL soma (for watershed seeding) and count cells
body_lab = label(cell_body_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cell bodies.")

# Watershed to partition the entire neuron into per-cell territories
if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

# (3) AXON (thin) via ridge + local threshold, inside neuron, excluding cell body
axon_candidate = np.zeros_like(ridge, dtype=bool)
for z in range(ridge.shape[0]):
    Rt = threshold_local(ridge[z], block_size=81, offset=-0.4*np.std(ridge[z]))
    axon_candidate[z] = ridge[z] > Rt
axon_candidate = remove_small_objects(axon_candidate, min_size=100, connectivity=3)
axon_candidate = binary_opening(axon_candidate, ball(1))
axon_candidate = binary_closing(axon_candidate, ball(1))

axon_mask = axon_candidate & neuron_mask & (~cell_body_mask)   # exclusive split

# (4) Optional: skeleton for the axon
axon_skel = skeletonize_3d(axon_mask)

# (5) Body shape scaffolding (largest soma slice)
axon_shape = "unknown"
if body_lab.max() > 0:
    props3d = regionprops(body_lab)
    props3d.sort(key=lambda p: p.area, reverse=True)
    lab = props3d[0].label
    z_areas = [np.sum(body_lab[z] == lab) for z in range(body_lab.shape[0])]
    z_best = int(np.argmax(z_areas))
    body_2d = (body_lab[z_best] == lab).astype(np.uint8)
    rp2 = regionprops(body_2d)
    # if rp2:
    #     ecc = rp2[0].eccentricity
    #     axon_shape = "circular" if ecc < 0.6 else "elliptical"

# Sanity
print("neuron voxels:", int(neuron_mask.sum()))
print("cell body voxels:", int(cell_body_mask.sum()))
print("axon voxels:", int(axon_mask.sum()))

# --------------------------
# 5) Map lysosomes to specific cell IDs and parts (axon/body)
# --------------------------
in_neuron = []
in_axon_ch2 = []
in_body_ch2 = []
location_ch2 = []
cell_id_list = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            in_neuron.append(False); in_axon_ch2.append(False); in_body_ch2.append(False)
            location_ch2.append("outside"); cell_id_list.append(0)
            continue

        n  = bool(neuron_mask[zz, yy, xx])
        ax = bool(axon_mask[zz, yy, xx])
        cb = bool(cell_body_mask[zz, yy, xx])
        cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0

        in_neuron.append(n)
        in_axon_ch2.append(ax)
        in_body_ch2.append(cb)

        if not n:
            location_ch2.append("outside")
            cell_id_list.append(0)
        else:
            if cb:
                location_ch2.append("cell body")
            elif ax:
                location_ch2.append("axon")
            else:
                location_ch2.append("neuron_other")
            cell_id_list.append(cid)

if len(df) > 0:
    df["in_neuron_ch2"] = in_neuron
    df["in_axon_ch2"]   = in_axon_ch2
    df["in_body_ch2"]   = in_body_ch2
    df["location_ch2"]  = location_ch2
    df["cell_id_ch2"]   = cell_id_list  # 0 = outside / unassigned

    # overall counts
    counts = pd.Series({
        "total_lysosomes": len(df),
        "in_neuron": int(np.sum(df["in_neuron_ch2"])),
        "in_axon":   int(np.sum(df["in_axon_ch2"])),
        "in_body":   int(np.sum(df["in_body_ch2"])),
    })
    counts.to_frame(name="value").to_csv("lysosome_location_summary.csv")

    # per-class counts
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_by_location.csv", index=False)

    # per-cell, per-part counts
    per_cell_counts = (
        df.groupby(["cell_id_ch2", "location_ch2"])
          .size()
          .reset_index(name="count")
          .sort_values(["cell_id_ch2", "location_ch2"])
    )
    per_cell_counts.to_csv("lysosome_counts_by_cell_and_part.csv", index=False)

    # flat summary per cell
    summary_per_cell = (
        df.pivot_table(index="cell_id_ch2",
                       columns="location_ch2",
                       values="id",
                       aggfunc="count",
                       fill_value=0)
        .reset_index()
        .rename(columns={"cell_id_ch2": "cell_id"})
    )
    summary_per_cell.to_csv("lysosome_summary_per_cell.csv", index=False)

    # Export full table
    df.to_csv("lysosomes_with_cell_ids.csv", index=False)

print("Saved: lysosome_location_summary.csv, lysosome_counts_by_location.csv, "
      "lysosome_counts_by_cell_and_part.csv, lysosome_summary_per_cell.csv, lysosomes_with_cell_ids.csv")

# --------------------------
# 6) Visualization
# --------------------------
# Axon (red)
axon_layer = viewer.add_labels(axon_mask.astype(np.uint8), name='Axon (Ch2)', opacity=0.35)
try:
    axon_layer.color = {1: (1.0, 0.0, 0.0, 1.0)}  # red
except Exception:
    pass
axon_layer.blending = 'translucent_no_depth'

# Cell body (green)
cellbody_layer = viewer.add_labels(cell_body_mask.astype(np.uint8), name='Cell body (Ch2)', opacity=0.35)
try:
    cellbody_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cellbody_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'

    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Axon skeleton (thin gray)
viewer.add_image(
    axon_skel.astype(np.uint8),
    name='Axon skeleton',
    blending='additive',
    contrast_limits=(0, 1),
    colormap='gray',
    opacity=0.8
)

# Lysosomes colored by location and labeled with cell_id_ch2
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist())
    colors = np.zeros((len(loc), 4), dtype=float)
    colors[loc == "axon"]         = [1.0, 0.5, 0.0, 1.0]  # orange
    colors[loc == "cell body"]    = [0.0, 1.0, 1.0, 1.0]  # cyan
    colors[loc == "outside"]      = [1.0, 1.0, 1.0, 1.0]  # white
    colors[loc == "neuron_other"] = [0.6, 0.6, 0.6, 1.0]  # gray

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        face_color=colors,
        name='Lysosomes'
    )
    try:
        pts.properties = {
            'lys_id': df['id'].to_numpy(),
            'where':  df['location_ch2'].to_numpy(),
            'cell':   df['cell_id_ch2'].to_numpy()
        }
        # Show cell_id_ch2 on each lysosome
        pts.text = {'text': 'C:{cell}', 'size': 10, 'color': 'white', 'anchor': 'upper left'}
        pts.edge_color = 'black'
        pts.edge_width = 0.3
    except Exception:
        pass

# >>> NEW: Big labels at each cell's 3D centroid <<<
if n_cells > 0:
    props = regionprops(cell_seg)   # one RegionProperties per label (>0)
    centroids = np.array([p.centroid for p in props], dtype=float)  # (z,y,x)
    ids = np.array([p.label for p in props], dtype=int)
    cell_pts = viewer.add_points(
        centroids,
        name='Cell ID labels',
        size=12,
        face_color='yellow'
        #edge_color='black',
        #edge_width=0.5,
        #opacity=0.9
    )
    # set styling AFTER creation for older napari APIs
    try:
        cell_pts.edge_color = 'black'
        cell_pts.edge_width = 0.5
        cell_pts.opacity = 0.9
        cell_pts.face_color = 'yellow'
        cell_pts.properties = {'cell': ids}
        cell_pts.text = {'text': 'C:{cell}', 'size': 14, 'color': 'yellow', 'anchor': 'center'}
    except Exception:
        pass

# Switch to 3D view
viewer.dims.ndisplay = 3
try:
    viewer.title = "Neuron segmentation body/axon (per-cell territories, 3D)"
except Exception:
    pass

# --------------------------
# 7) Quick fused 2D video (optional, unchanged)
# --------------------------
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    axon = (axon_mask[z].astype(np.uint8) * 255)
    body = (cell_body_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 2] = np.maximum(overlay[..., 2], axon)  # red for axon
    overlay[..., 1] = np.maximum(overlay[..., 1], body)  # green for cell body
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < axon_mask.shape[1] and 0 <= x < axon_mask.shape[2] and axon_mask[z, y, x]:
                color = (0, 128, 255)  # orange-ish (axon)
            elif 0 <= y < cell_body_mask.shape[1] and 0 <= x < cell_body_mask.shape[2] and cell_body_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell body)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_axon_cellbody.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_axon_cellbody.mp4")
except Exception:
    imageio.mimsave('ch2_fused_axon_cellbody.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_axon_cellbody.gif")

# --------------------------
# 8) 3D snapshot + rotation video (POST-SEGMENTATION)
# --------------------------
def save_3d_outputs(viewer,
                    still_path='segmentation_3d_snapshot.png',
                    video_path='segmentation_3d_spin.mp4',
                    elev=30, azimuth_start=0, azimuth_end=360, step=2, fps=20):
    """Save one 3D screenshot and a rotation movie of the current napari scene."""
    viewer.dims.ndisplay = 3
    # Initial view
    try:
        viewer.camera.angles = (elev, azimuth_start, 0)
    except Exception:
        try:
            viewer.camera.elevation = elev
            viewer.camera.azimuth = azimuth_start
        except Exception:
            pass

    # Still image
    img = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(still_path, img)
    except Exception:
        cv2.imwrite(still_path, cv2.cvtColor(img, cv2.COLOR_RGBA2BGRA))
    print(f"Saved: {still_path}")

    # Spin & record
    frames = []
    for az in range(azimuth_start, azimuth_end, step):
        try:
            viewer.camera.angles = (elev, az, 0)
        except Exception:
            try:
                viewer.camera.azimuth = az
                viewer.camera.elevation = elev
            except Exception:
                pass
        # small delay helps rendering (esp. on some GPUs)
        time.sleep(0.01)
        frames.append(viewer.screenshot(canvas_only=True))

    try:
        imageio.mimsave(video_path, frames, fps=fps, format='FFMPEG')
        print(f"Saved: {video_path}")
    except Exception:
        gif_path = video_path.rsplit('.', 1)[0] + '.gif'
        imageio.mimsave(gif_path, frames, fps=fps)
        print(f"FFMPEG not available, saved GIF instead: {gif_path}")

# Create the 3D outputs now (before entering the event loop)
save_3d_outputs(viewer,
                still_path='segmentation_3d_snapshot.png',
                video_path='segmentation_3d_spin.mp4',
                elev=30, azimuth_start=0, azimuth_end=360, step=2, fps=20)

# --------------------------
# 9) Run viewer
# --------------------------
napari.run()

In [ ]:
####VERSION 3############################

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, meijering, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import time

from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, skeletonize_3d
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_2_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (axon + cell body)

# --------------------------
# 2) CH1: blobs (lysosomes)
# --------------------------
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# Per-blob DF & region bins (optional export kept)
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("volume_voxels", "sum")
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 3) Viewer base
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 4) CH2: neuron segmentation (axon = thin, cell body = thick)  ✅ three-way
# --------------------------
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)

# Thin-process ridge response (neurites)
ridge = meijering(ch2, sigmas=np.linspace(0.6, 2.0, 8), black_ridges=False)

# (1) Base NEURON mask (axon + body) via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=81, offset=-0.4*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(3))

# (2) CELL BODY (thick) via distance transform
dist = edt(neuron_mask)
cell_body_min_radius_vox = 1   # tune (e.g., 4–6) for thicker soma
cell_body_mask = (dist >= cell_body_min_radius_vox)
cell_body_mask &= neuron_mask
cell_body_mask = binary_opening(cell_body_mask, ball(3))
cell_body_mask = binary_closing(cell_body_mask, ball(4))

# Label ALL soma (for watershed seeding) and count cells
body_lab = label(cell_body_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cell bodies.")

# Watershed to partition the entire neuron into per-cell territories
if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

# (3) AXON (thin) — EXHAUSTIVE 3-way split
# You can still compute axon_candidate from ridge for QA, but final axon is complement in neuron.
axon_candidate = np.zeros_like(ridge, dtype=bool)
for z in range(ridge.shape[0]):
    Rt = threshold_local(ridge[z], block_size=81, offset=-0.4*np.std(ridge[z]))
    axon_candidate[z] = ridge[z] > Rt
axon_candidate = remove_small_objects(axon_candidate, min_size=100, connectivity=3)
axon_candidate = binary_opening(axon_candidate, ball(1))
axon_candidate = binary_closing(axon_candidate, ball(1))

# FINAL axon mask: everything in neuron that is NOT cell body
axon_mask = neuron_mask & (~cell_body_mask)

# (4) Optional: skeleton for the axon
axon_skel = skeletonize_3d(axon_mask)

# (5) Build the exclusive 3-way labels: 0=outside, 1=cell body, 2=cell axon
three_way = np.zeros_like(neuron_mask, dtype=np.uint8)
three_way[cell_body_mask] = 1
three_way[axon_mask]      = 2

# Save the 3-way volume
try:
    import tifffile as tiff
    tiff.imwrite('segmentation_three_way.tif', three_way.astype(np.uint8))
    print("Saved: segmentation_three_way.tif")
except Exception:
    np.save('segmentation_three_way.npy', three_way.astype(np.uint8))
    print("Saved: segmentation_three_way.npy")

# >>> NEW: Per-cell 3-way volume (0=outside; cell_id*10+1=body; cell_id*10+2=axon)
percell_three = np.zeros_like(cell_seg, dtype=np.int32)
inside_mask = three_way > 0
percell_three[inside_mask] = (
    cell_seg[inside_mask].astype(np.int32) * 10 + three_way[inside_mask].astype(np.int32)
)
# Save it
try:
    import tifffile as tiff
    tiff.imwrite('segmentation_percell_three_way.tif', percell_three.astype(np.int32))
    print("Saved: segmentation_percell_three_way.tif")
except Exception:
    np.save('segmentation_percell_three_way.npy', percell_three.astype(np.int32))
    print("Saved: segmentation_percell_three_way.npy")
# Decode rule for later: cell_id = val // 10 ; class = val % 10 (1=body, 2=axon)

# (6) Body shape scaffolding (largest soma slice)
axon_shape = "unknown"
if body_lab.max() > 0:
    props3d = regionprops(body_lab)
    props3d.sort(key=lambda p: p.area, reverse=True)
    lab = props3d[0].label
    z_areas = [np.sum(body_lab[z] == lab) for z in range(body_lab.shape[0])]
    z_best = int(np.argmax(z_areas))
    body_2d = (body_lab[z_best] == lab).astype(np.uint8)
    rp2 = regionprops(body_2d)
    # if rp2:
    #     ecc = rp2[0].eccentricity
    #     axon_shape = "circular" if ecc < 0.6 else "elliptical"

# Sanity
print("neuron voxels:", int(neuron_mask.sum()))
print("cell body voxels:", int(cell_body_mask.sum()))
print("axon voxels:", int(axon_mask.sum()))

# --------------------------
# 5) Map lysosomes to 3 classes (cell body / cell axon / outside), with per-cell IDs
# --------------------------
labels_for_lys = []   # "cell body" | "cell axon" | "outside"
cell_id_list  = []    # from watershed territories (0 if outside/unassigned)

if len(blobs) > 0:
    Z, Y, X = three_way.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            labels_for_lys.append("outside"); cell_id_list.append(0); continue

        lab = int(three_way[zz, yy, xx])  # 0,1,2
        if lab == 1:
            labels_for_lys.append("cell body")
        elif lab == 2:
            labels_for_lys.append("cell axon")
        else:
            labels_for_lys.append("outside")

        cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
        cell_id_list.append(cid)

# attach to df
if len(df) > 0:
    df["location_3way"] = labels_for_lys
    df["cell_id_ch2"]   = cell_id_list

    # Per-class counts (3 classes only)
    df.groupby("location_3way").size().reset_index(name="count") \
      .to_csv("lysosome_counts_3way.csv", index=False)

    # Per-cell x class
    per_cell_counts_3way = (
        df.groupby(["cell_id_ch2", "location_3way"])
          .size().reset_index(name="count")
          .sort_values(["cell_id_ch2", "location_3way"])
    )
    per_cell_counts_3way.to_csv("lysosome_counts_by_cell_3way.csv", index=False)

    # Full table
    df.to_csv("lysosomes_with_cell_ids_3way.csv", index=False)

print("Saved: lysosome_counts_3way.csv, lysosome_counts_by_cell_3way.csv, lysosomes_with_cell_ids_3way.csv")

# --------------------------
# 6) Visualization
# --------------------------
# Axon (red)
axon_layer = viewer.add_labels(axon_mask.astype(np.uint8), name='Axon (Ch2)', opacity=0.35)
try:
    axon_layer.color = {1: (1.0, 0.0, 0.0, 1.0)}  # red
except Exception:
    pass
axon_layer.blending = 'translucent_no_depth'

# Cell body (green)
cellbody_layer = viewer.add_labels(cell_body_mask.astype(np.uint8), name='Cell body (Ch2)', opacity=0.35)
try:
    cellbody_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cellbody_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'

    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Axon skeleton (thin gray)
viewer.add_image(
    axon_skel.astype(np.uint8),
    name='Axon skeleton',
    blending='additive',
    contrast_limits=(0, 1),
    colormap='gray',
    opacity=0.8
)

# 3-way labels (0=outside transparent, 1=body green, 2=axon red)
three_layer = viewer.add_labels(three_way.astype(np.uint8), name='3-way (body/axon/outside)', opacity=0.5)
try:
    three_layer.color = {
        0: (0.0, 0.0, 0.0, 0.0),  # outside transparent
        1: (0.0, 1.0, 0.0, 1.0),  # cell body = green
        2: (1.0, 0.0, 0.0, 1.0),  # cell axon = red
    }
except Exception:
    pass
three_layer.blending = 'translucent_no_depth'

# >>> NEW: Per-cell 3-way labels layer (auto-colored per unique code)
percell_layer = viewer.add_labels(
    percell_three.astype(np.int32),
    name='Per-cell 3-way (cell*10+class)',
    opacity=0.45
)
percell_layer.blending = 'translucent_no_depth'
# Optional boundaries for this composite label map
percell_bound = find_boundaries(percell_three, connectivity=1, mode='outer')
viewer.add_image(
    percell_bound.astype(np.uint8),
    name='Per-cell 3-way boundaries',
    blending='additive',
    contrast_limits=(0, 1),
    colormap='cyan',
    opacity=0.5
)

# Lysosomes colored by 3-way location + labeled with cell_id_ch2
if len(blobs) > 0:
    loc = np.array(df["location_3way"].tolist())
    colors = np.zeros((len(loc), 4), dtype=float)
    colors[loc == "cell axon"] = [1.0, 0.5, 0.0, 1.0]  # orange for axon
    colors[loc == "cell body"] = [0.0, 1.0, 1.0, 1.0]  # cyan for body
    colors[loc == "outside"]   = [1.0, 1.0, 1.0, 1.0]  # white for outside

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        face_color=colors,
        name='Lysosomes (3-way)'
    )
    try:
        pts.properties = {
            'lys_id': df['id'].to_numpy(),
            'where':  df['location_3way'].to_numpy(),
            'cell':   df['cell_id_ch2'].to_numpy()
        }
        pts.text = {'text': 'C:{cell} {where}', 'size': 10, 'color': 'white', 'anchor': 'upper left'}
        pts.edge_color = 'black'
        pts.edge_width = 0.3
    except Exception:
        pass

# Big labels at each cell's 3D centroid
if n_cells > 0:
    props = regionprops(cell_seg)   # one RegionProperties per label (>0)
    centroids = np.array([p.centroid for p in props], dtype=float)  # (z,y,x)
    ids = np.array([p.label for p in props], dtype=int)
    cell_pts = viewer.add_points(
        centroids,
        name='Cell ID labels',
        size=12,
        face_color='yellow',
        #edge_color='black',
        #edge_width=0.5,
        #opacity=0.9
    )
    # set styling AFTER creation for older napari APIs
    try:
        cell_pts.edge_color = 'black'
        cell_pts.edge_width = 0.5
        cell_pts.opacity = 0.9
        cell_pts.face_color = 'yellow'
        cell_pts.properties = {'cell': ids}
        cell_pts.text = {'text': 'C:{cell}', 'size': 14, 'color': 'yellow', 'anchor': 'center'}
    except Exception:
        pass

# Switch to 3D view
viewer.dims.ndisplay = 3
try:
    viewer.title = "Neuron 3-way segmentation (soma/axon/outside) + Per-cell 3-way"
except Exception:
    pass

# --------------------------
# 7) Quick fused 2D video (optional)
# --------------------------
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    axon = (axon_mask[z].astype(np.uint8) * 255)
    body = (cell_body_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 2] = np.maximum(overlay[..., 2], axon)  # red for axon
    overlay[..., 1] = np.maximum(overlay[..., 1], body)  # green for cell body
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < axon_mask.shape[1] and 0 <= x < axon_mask.shape[2] and axon_mask[z, y, x]:
                color = (0, 128, 255)  # orange-ish (axon)
            elif 0 <= y < cell_body_mask.shape[1] and 0 <= x < cell_body_mask.shape[2] and cell_body_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell body)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_axon_cellbody.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_axon_cellbody.mp4")
except Exception:
    imageio.mimsave('ch2_fused_axon_cellbody.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_axon_cellbody.gif")

# --------------------------
# 8) 3D snapshot + rotation video (POST-SEGMENTATION)
# --------------------------
def save_3d_outputs(viewer,
                    still_path='segmentation_3d_snapshot.png',
                    video_path='segmentation_3d_spin.mp4',
                    elev=30, azimuth_start=0, azimuth_end=360, step=2, fps=20):
    """Save one 3D screenshot and a rotation movie of the current napari scene."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (elev, azimuth_start, 0)
    except Exception:
        try:
            viewer.camera.elevation = elev
            viewer.camera.azimuth = azimuth_start
        except Exception:
            pass

    # Still image
    img = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(still_path, img)
    except Exception:
        cv2.imwrite(still_path, cv2.cvtColor(img, cv2.COLOR_RGBA2BGRA))
    print(f"Saved: {still_path}")

    # Spin & record
    frames = []
    for az in range(azimuth_start, azimuth_end, step):
        try:
            viewer.camera.angles = (elev, az, 0)
        except Exception:
            try:
                viewer.camera.azimuth = az
                viewer.camera.elevation = elev
            except Exception:
                pass
        time.sleep(0.01)
        frames.append(viewer.screenshot(canvas_only=True))

    try:
        imageio.mimsave(video_path, frames, fps=fps, format='FFMPEG')
        print(f"Saved: {video_path}")
    except Exception:
        gif_path = video_path.rsplit('.', 1)[0] + '.gif'
        imageio.mimsave(gif_path, frames, fps=fps)
        print(f"FFMPEG not available, saved GIF instead: {gif_path}")

# Create the 3D outputs now (before entering the event loop)
save_3d_outputs(viewer,
                still_path='segmentation_3d_snapshot.png',
                video_path='segmentation_3d_spin.mp4',
                elev=30, azimuth_start=0, azimuth_end=360, step=2, fps=20)

# --------------------------
# 9) Run viewer
# --------------------------
napari.run()